In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image

In [ ]:
desired_size = (256, 256)

In [ ]:
import torch
from torchvision import io, transforms

# Définir la transformation de redimensionnement avec l'interpolation 'nearest'
resize_transform = transforms.Resize(desired_size, interpolation=transforms.InterpolationMode.NEAREST)

transform = transforms.Compose([
    transforms.ToTensor()  # Convertit l'image PIL en tenseur PyTorch
]) # normalise directement entre 0 et 1

In [ ]:
from PIL import Image

def resize_and_pad(image_path, target_size=(256, 256)):
    # Charger l'image
    with Image.open(image_path) as img:

        if img.mode == "L" or img.mode == "CMYK":
            img = img.convert('RGB')

        # Redimensionner l'image en conservant le rapport d'aspect
        img.thumbnail(target_size, Image.Resampling.LANCZOS)

        # Créer une nouvelle image avec la taille cible et un fond noir
        new_img = Image.new("RGB", target_size, (0, 0, 0))

        # Calculer les coordonnées pour centrer l'image redimensionnée
        left = (target_size[0] - img.size[0]) // 2
        top = (target_size[1] - img.size[1]) // 2

        # Coller l'image redimensionnée sur la nouvelle image
        new_img.paste(img, (left, top))

        new_img_tensor = transform(new_img)

        return new_img, new_img_tensor

---

MODEL

In [ ]:
import torchvision.models as models
import torch.nn as nn

model = models.resnet18(weights=True)
num_features = model.fc.in_features
dropout_rate = 0.5  # Par exemple, 50% de chance de désactiver chaque neurone
model.fc = nn.Sequential(
    nn.Dropout(dropout_rate),  # Ajouter la couche de dropout
    nn.Linear(num_features, 1)  # Couche finale pour la classification binaire
) # 1 for the binary classification

In [ ]:
model.load_state_dict(torch.load("best_model.pth"))
model.eval() # I forgot it and that caused me some issues!

---

In [ ]:
import os

directory_path = 'images'
file_names = []

nb_images = 0
for _, _, files in os.walk(directory_path):
    nb_images += len(files)
    for file in files:
        file_names.append(file)

In [ ]:
nb_images
file_names

In [ ]:
fig, axes = plt.subplots(nb_images, figsize=(16, 16))

ax = axes.flatten()
count = 0

for file in file_names:
    ax[count].axis('off')
    image, image_tensor = resize_and_pad(f"images/{file}")
    
    image_tensor = image_tensor.unsqueeze(0)

    proba = model(image_tensor)
    proba = torch.sigmoid(proba)

    if proba > 0.5:
        prediction = "Dog"
    else:
        prediction = "Cat"
        proba[0][0] = 1 - proba[0][0]

    ax[count].set_title(f'This is a {prediction} with {proba.item()*100:.2f}%')

    ax[count].imshow(image)

    count += 1